## Optimization using gradient descent

This notebook demonstrates multivariate linear fits using the gradient descent method.


In [1]:
%matplotlib widget

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm

import sys
sys.path.append('/Users/ondrea/MLandstats/OStats/')
from ostats import ML
from ostats import dplot as dp
#from mpltools import color as colors

In [2]:
data_range = np.random.RandomState(1) #make up some fake data for testing
fakex = 10 * data_range.rand(50)
fakey = 3 * fakex - 5 + data_range.randn(50)


!rm -rf ./data/housing*
!wget -P ./data https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv
    
#get some data for testing
!wget -P ./data https://datahub.io/core/global-temp/r/annual.csv

--2021-06-14 11:47:21--  https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘./data/housing.csv’

housing.csv         100%[===================>]   1.36M  --.-KB/s    in 0.08s   

2021-06-14 11:47:22 (16.4 MB/s) - ‘./data/housing.csv’ saved [1423529/1423529]

--2021-06-14 11:47:22--  https://datahub.io/core/global-temp/r/annual.csv
Resolving datahub.io (datahub.io)... 104.21.40.221, 172.67.157.38
Connecting to datahub.io (datahub.io)|104.21.40.221|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://pkgstore.datahub.io/core/global-temp/annual_csv/data/a26b154688b061cdd04f1df36e4408be/annual_csv.csv [fol

In [3]:
year, tempa = np.loadtxt('./data/annual.csv', skiprows=1, delimiter=",", usecols=(1,2), unpack=True)
year = np.array([int(y) for y in year])/np.max(year); tempa=np.array(tempa)+1

In [4]:
year = 1 + (year - np.mean(year))/(np.max(year)-np.min(year))

In [5]:
#year

In [6]:
housing_data = pd.read_csv('./data/housing.csv')

In [7]:
housing_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [8]:
housing_data = housing_data[housing_data.median_house_value != 500001] #a bunch of values are set to this
housing_data = housing_data.sample(n=250) #smaller sample
housing_data.median_house_value = housing_data.median_house_value/500000
housing_data.median_income= housing_data.median_income/10

In [9]:
housing_data.median_income.max()

0.8540699999999999

In [10]:
housing_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
3289,-122.69,39.02,27.0,2199.0,527.0,744.0,316.0,0.21094,0.1448,INLAND
13397,-117.53,34.06,18.0,5605.0,1303.0,4028.0,1145.0,0.29386,0.2328,INLAND
11797,-121.31,38.97,16.0,1210.0,228.0,726.0,222.0,0.27083,0.1642,INLAND
6998,-118.04,33.95,36.0,2722.0,515.0,1390.0,486.0,0.38214,0.3570,<1H OCEAN
2363,-119.58,36.77,19.0,3225.0,548.0,1760.0,542.0,0.40227,0.2530,INLAND
...,...,...,...,...,...,...,...,...,...,...
17480,-119.86,34.38,26.0,1626.0,375.0,1580.0,359.0,0.21471,0.3750,NEAR OCEAN
4983,-118.30,34.00,52.0,1686.0,377.0,982.0,356.0,0.20958,0.2328,<1H OCEAN
14454,-117.27,32.83,39.0,1877.0,426.0,805.0,409.0,0.38750,0.8200,NEAR OCEAN
7322,-118.17,33.98,27.0,1871.0,556.0,2542.0,581.0,0.28427,0.3288,<1H OCEAN


In [11]:
x_house = housing_data['median_house_value'].to_numpy()
y_house = housing_data['median_income'].to_numpy()

In [12]:
print(x_house)

[0.1448 0.2328 0.1642 0.357  0.253  0.4096 0.4338 0.3034 0.9    0.4774
 0.6834 0.725  0.4252 0.2652 0.2842 0.725  0.67   0.3074 0.7508 0.2274
 0.225  0.2482 0.5864 0.4628 0.917  0.14   0.259  0.1878 0.794  0.3876
 0.2864 0.1476 0.4116 0.2172 0.3    0.4524 0.691  0.3248 0.4786 0.3476
 0.135  0.4714 0.274  0.3384 0.869  0.3056 0.1672 0.503  0.088  0.86
 0.17   0.1738 0.4924 0.5716 0.268  0.3326 0.4292 0.3892 0.7326 0.1688
 0.1408 0.4068 0.5218 0.1372 0.7    0.604  0.668  0.2724 0.1392 0.3386
 0.0866 0.3404 0.0992 0.3844 0.7742 0.1804 0.2212 0.3986 0.2328 0.1436
 0.4182 0.2838 0.196  0.2578 0.4538 0.2316 0.1886 0.3736 0.5524 0.6372
 0.154  0.7434 0.3296 0.3154 0.2068 0.2822 0.275  0.5254 0.8694 0.1794
 0.3282 0.3288 0.2458 0.2152 0.325  0.11   0.3964 0.4346 0.2562 0.8052
 0.4232 0.2908 0.329  0.2528 0.619  0.4716 0.4978 0.1888 0.6066 0.3
 0.3836 0.2946 0.2548 0.335  0.295  0.6318 0.2682 0.1568 0.4    0.3968
 0.7038 0.2844 0.7122 0.3228 0.1918 0.2104 0.1266 0.375  0.25   0.1214
 0.3072 0.1

In [13]:
def plotfit(x, beta):
    funct = beta[-1][0]+x*beta[-1][1]
    return(funct)

def plotfit2(x, beta):
    functs=[]
    points = np.linspace(0,len(beta),10)
    points = [int(x) for x in points]
    for i in range(len(points)-1):
        functs.append(beta[points[i]][0]+x*beta[points[i]][1])
    return(functs)

### Fake data

In [14]:
ifig=1;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
theta_fake, J_fake = ML.GradDes_Regression(x=fakex, y=fakey, gamma=0.01)
plt.scatter(fakex,fakey, c='darkred',  s=15)
nlines = 10
for i in range(nlines):
    plt.plot(fakex,plotfit2(fakex,beta=theta_fake)[i-1], color=dp.ColorGradient(stop=(11, 0, 59), n=nlines)[i],lw=1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/ondrea/MLandstats/OStats/ostats/ML.py:40: RuntimeWarning: divide by zero encountered in double_scalars
  stop_condition = (np.abs(theta[1] - store_theta[-1][1])/store_theta[-1][1] < epsilon) and (np.abs(theta[0] -\


### Housing data

In [122]:
theta_house_normal=ML.Normal_Linear_Regression(x=x_house, y=y_house)
ynormal = np.array(theta_house_normal[1])*x_house + np.array(theta_house_normal[0])

theta_house, J_house = ML.GradDes_Regression(x=x_house, y=y_house, gamma=1e-2, epsilon = 1e-6, theta_init = np.array([0.23,0.37] ))
theta_house_alt, J_house_alt = ML.GradDes_Regression(x=x_house, y=y_house, gamma=1e-3,epsilon = 1e-5, theta_init = np.array([0.5,0.5]))
y = np.array(theta_house[-1][1])*x_house + np.array(theta_house[-1][0])
yalt = np.array(theta_house_alt[-1][1])*x_house + np.array(theta_house_alt[-1][0])

In [129]:
ifig=2;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)

plt.scatter(x_house, y=y_house, c='g',  s=15, label='data')
plt.plot(x_house,ynormal, c='b', label='Analytic', lw=3)
nlines = 10
#for i in range(nlines):
#    plt.plot(x_house,plotfit2(x_house,beta=theta_house)[i-1],\
#             color=dp.ColorGradient(stop=(0, 200, 90), n=nlines)[i],lw=1)
#    plt.plot(x_house,plotfit2(x_house,beta=theta_house_alt)[i-1],\
#             color=dp.ColorGradient(stop=(140, 20, 90), n=nlines)[i],lw=1)

plt.plot(x_house,y, c='k', label='gamma=1e-2, epsilon = 1e-6')
plt.plot(x_house,yalt, c='r', label = 'gamma=1e-3,epsilon = 1e-5')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
### Climate data
theta_climate, J_climate = ML.GradDes_Regression(x=year, y=tempa, gamma=0.01, epsilon=0.0001, theta_init = np.array([0.5,0.5]))
theta_climate_normal=ML.Normal_Linear_Regression(x=year, y=tempa)
ynormal_climate = np.array(theta_climate_normal[1])*year + np.array(theta_climate_normal[0])

In [18]:
ifig=3;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)

plt.scatter(year, y=tempa, c='orange',  s=15)
plt.plot(year,ynormal_climate, c='b')
nlines = 10
for i in range(nlines):
    plt.plot(year,plotfit2(year,beta=theta_climate)[i-1],\
             color=dp.ColorGradient(stop=(31, 80, 40), n=nlines)[i],lw=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Now take a look at convergence by plotting $J(\theta)$ vs $(\theta)$ or $J(\theta)$ vs n iter

In [104]:
theta_house, J_house = ML.GradDes_Regression(x=x_house, y=y_house, gamma=0.001, epsilon = 0.000001, theta_init = np.array([0.23,0.37] ))
theta_house_alt, J_house_alt = ML.GradDes_Regression(x=x_house, y=y_house, gamma=0.001,epsilon = 0.00001, theta_init = np.array([0.5,0.5]))

In [105]:
#For the two instances of housing data
ifig=4;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
theta0 = [x[0] for x in theta_house]
theta0_alt = [x[0] for x in theta_house_alt]
theta1 = [x[1] for x in theta_house]
theta1_alt = [x[1] for x in theta_house_alt]

niter = range(len(theta0))
niter2 = range(len(theta0_alt))

plt.plot(J_house, theta0, color = 'k')
plt.plot(J_house_alt, theta0_alt, marker='.', ls='',color='k', alpha=0.2)
plt.plot(J_house, theta1, color = 'r')
plt.plot(J_house_alt, theta1_alt, marker='.',ls='', color = 'r', alpha=0.1)
plt.xlabel('J')
plt.ylabel('theta')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'theta')

In [106]:
ifig=5;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
plt.plot(niter, J_house, color = 'b', lw=4)
plt.plot(niter2, J_house_alt, marker='_', color='k')
plt.xlabel('Iterations')
plt.ylabel('J')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'J')

In [107]:
print(J_house[-1],J_house_alt[-1])


3.677570385404085 3.696615819248713


In [108]:
#For the two instances of housing data
ifig=6;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
plt.plot(theta0, theta1, c='b', lw=3)
plt.plot(theta0_alt, theta1_alt, c='y')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [109]:
ifig=7;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
plt.plot(niter,theta0, label='theta0')
plt.plot(niter,theta1, label='theta1 alt')

plt.plot(niter2,theta0_alt, label='theta0 alt')
plt.plot(niter2,theta1_alt, label='theta1 alt')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [110]:
def cost_func(theta0, theta1,x,y):
    h = theta0 + theta1*x
    theta0 = np.atleast_3d(np.asarray(theta0))
    theta1 = np.atleast_3d(np.asarray(theta1))
    return np.average((y-h)**2, axis=2)/2

In [111]:
th0,th1 = np.linspace(0, 1.0, len(x_house)),np.linspace(0, 1.0, len(x_house))

J_grid = cost_func(th0[np.newaxis,:,np.newaxis],
                   th1[:,np.newaxis,np.newaxis],x_house, y_house)
#th0,th1 = np.meshgrid(th0,th1)

In [112]:
ifig=8;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
color_bar_idxs = np.linspace(0,len(theta0),len(theta0))
color_bar_idxs_alt = np.linspace(0,len(theta0_alt),len(theta0_alt))

plt.contourf(th0,th1, J_grid)
plt.scatter(theta0, theta1, c= color_bar_idxs, cmap ='magma', s =1)
plt.scatter(theta0_alt, theta1_alt, c= color_bar_idxs_alt, cmap ='cividis', s =1)
plt.scatter(theta_house_normal[0], theta_house_normal[1], c='k')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …